# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#Use jupyter-gmaps and the Google Places API for this part of the assignment.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

#print(g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv("./output_data/cities.csv")
cities_df.head()


,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,clyde river,70.4692,-68.5914,8.60,92,75,24.16,CA,1611543211
1,vaini,-21.2000,-175.2000,82.40,83,75,9.22,TO,1611543211
2,barrow,71.2906,-156.7887,-5.80,77,1,11.50,US,1611543212
3,hermanus,-34.4187,19.2345,64.99,78,6,13.00,ZA,1611543212
4,busselton,-33.6500,115.3333,77.00,50,29,1.99,AU,1611543212


In [4]:
#Create a heat map that displays the humidity for every city from Part I.


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
locations = cities_df[["Lat","Lng"]]

locations

,Lat,Lng
0,70.4692,-68.5914
1,-21.2000,-175.2000
2,71.2906,-156.7887
3,-34.4187,19.2345
4,-33.6500,115.3333
...,...,...
553,16.1307,119.7824
554,30.4833,-115.9500
555,54.2693,41.4465
556,57.5058,-1.7981


In [6]:
# Plot Heatmap
humidity = cities_df["Humidity"]
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=10, point_radius =1)
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
#Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.


In [8]:
# Filter for all rows for which the max temperature is lower than 80 but higher than 70
#Wind speed less than 10 mph
#less than 10% cloudiness
#less than 85% humidity
 
ideal_cities_df = cities_df[(cities_df['Temp'] <= 80) & (cities_df['Temp'] >= 70) & (cities_df['Wind Speed'] < 10) & (cities_df['Cloudiness'] < 10) & (cities_df['Humidity'] < 85)]
ideal_cities_df['Hotel Name'] = ""  

# Print the new dataframe 
ideal_cities_df


<ipython-input-8-a2ac149decc9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_cities_df['Hotel Name'] = ""


,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
63,pergamino,-33.8900,-60.5736,79.90,64,0,3.00,AR,1611543224,
89,chiang mai,18.7500,99.0000,70.11,68,0,2.30,TH,1611543101,
95,sao filipe,14.8961,-24.4956,71.37,67,0,5.84,CV,1611543232,
108,saint-pierre,-21.3393,55.4781,75.94,73,0,3.44,RE,1611543234,
117,palasa,18.7667,84.4167,75.81,67,7,4.50,IN,1611543236,
169,saint-philippe,-21.3585,55.7679,75.83,73,0,3.44,RE,1611543248,
308,necochea,-38.5473,-58.7368,78.01,58,7,8.01,AR,1611543279,
406,siddapur,14.3333,74.8833,71.35,48,0,5.66,IN,1611543300,
529,bonthe,7.5264,-12.5050,75.60,84,0,3.09,SL,1611543329,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
from api_keys import g_key

# geocoordinates
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

for index, row in ideal_cities_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    params["location"] = f"{lat},{lng}" 
   
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    hotel_data = response.json()
    #print(hotel_data)
    try:
        ideal_cities_df.loc[index, 'Hotel Name'] = hotel_data['results'][0]['name']
    except (KeyError, IndexError):
        print("Missing hotel")
ideal_cities_df

C:\Users\Cameron\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
63,pergamino,-33.8900,-60.5736,79.90,64,0,3.00,AR,1611543224,Hotel Fachinat
89,chiang mai,18.7500,99.0000,70.11,68,0,2.30,TH,1611543101,Holiday Inn Chiangmai
95,sao filipe,14.8961,-24.4956,71.37,67,0,5.84,CV,1611543232,Tortuga B&B
108,saint-pierre,-21.3393,55.4781,75.94,73,0,3.44,RE,1611543234,Lindsey Hôtel
117,palasa,18.7667,84.4167,75.81,67,7,4.50,IN,1611543236,Balaji tifen & meals veg & non veg hotel
169,saint-philippe,-21.3585,55.7679,75.83,73,0,3.44,RE,1611543248,"Chambres d'hôte ""La Trinité"""
308,necochea,-38.5473,-58.7368,78.01,58,7,8.01,AR,1611543279,dyd -mar Hotel
406,siddapur,14.3333,74.8833,71.35,48,0,5.66,IN,1611543300,Ibbani Homestay
529,bonthe,7.5264,-12.5050,75.60,84,0,3.09,SL,1611543329,Bonthe Holiday Village


In [10]:
#Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
#Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_cities_df.iterrows()]
locations = ideal_cities_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))